In [ ]:
import numpy as np

# --- Funciones Auxiliares (Basadas en Clase_NNA3) ---
def sig(s):
    # Función de activación sigmoide
    return 1 / (1 + np.exp(-s))

def dSig(s):
    # Derivada de la función sigmoide
    # s debe ser un vector/matriz de activaciones ya calculadas o potenciales
    # Ajustado para que funcione con la salida de sig(Z) o Z directamente si se maneja con cuidado
    # En el original: df = sig(s)*(sig(s)), pero dSig se suele aplicar a sig(z).
    # Usaremos la forma: f'(z) = f(z)(1-f(z)) si s es A, o sig(s)(1-sig(s)) si s es Z.
    # Siguiendo tu código original, se pasaba Z.
    val = sig(s)
    return val * (1 - val) # Derivada estándar: sigmoid(x) * (1 - sigmoid(x))

def error(A_final, Yd):
    return Yd - A_final

# --- Clase Red Neuronal con 2 Capas Ocultas ---
class RedNeuronalDosOcultas:
    def __init__(self, nn_input, nn_hidden1, nn_hidden2, nn_output, eta=0.5):
        self.eta = eta

        # Inicialización aleatoria de pesos y sesgos para 3 conjuntos de conexiones
        # Capa 1: Entrada -> Oculta 1
        self.W1 = np.random.uniform(-1, 1, (nn_hidden1, nn_input))
        self.b1 = np.random.uniform(-1, 1, (nn_hidden1, 1))

        # Capa 2: Oculta 1 -> Oculta 2
        self.W2 = np.random.uniform(-1, 1, (nn_hidden2, nn_hidden1))
        self.b2 = np.random.uniform(-1, 1, (nn_hidden2, 1))

        # Capa 3: Oculta 2 -> Salida
        self.W3 = np.random.uniform(-1, 1, (nn_output, nn_hidden2))
        self.b3 = np.random.uniform(-1, 1, (nn_output, 1))

    def propaga(self, X):
        # Capa 1
        self.Z1 = np.dot(self.W1, X) + self.b1
        self.A1 = sig(self.Z1)

        # Capa 2
        self.Z2 = np.dot(self.W2, self.A1) + self.b2
        self.A2 = sig(self.Z2)

        # Capa 3 (Salida)
        self.Z3 = np.dot(self.W3, self.A2) + self.b3
        self.A3 = sig(self.Z3)

        return self.A3

    def backpropagation(self, X, Yd):
        # El error es Yd - A3
        err = error(self.A3, Yd)

        # --- Retropropagación Capa 3 (Salida) ---
        # delta3 = dSig(Z3) * error. (Nota: en tu código usabas dSig con matriz diagonal, simplificamos con multiplicación elemento a elemento que es equivalente para vectores)
        delta3 = err * dSig(self.Z3)
        dEdW3 = -np.dot(delta3, self.A2.T)
        dEdb3 = -delta3

        # --- Retropropagación Capa 2 (Oculta 2) ---
        # El error se propaga hacia atrás a través de W3
        delta2 = np.dot(self.W3.T, delta3) * dSig(self.Z2)
        dEdW2 = -np.dot(delta2, self.A1.T)
        dEdb2 = -delta2

        # --- Retropropagación Capa 1 (Oculta 1) ---
        # El error se propaga hacia atrás a través de W2
        delta1 = np.dot(self.W2.T, delta2) * dSig(self.Z1)
        dEdW1 = -np.dot(delta1, X.T)
        dEdb1 = -delta1

        return dEdW1, dEdb1, dEdW2, dEdb2, dEdW3, dEdb3

    def train(self, X_train, Y_train, epochs=10000):
        for i in range(epochs):
            # Seleccionamos un ejemplo aleatorio o iteramos todos (aquí iteramos todos)
            # Para generalizar, usaremos Stochastic Gradient Descent (uno a uno)
            idx = np.random.randint(0, X_train.shape[1])
            X_sample = X_train[:, [idx]] # Mantiene dimensión columna (n, 1)
            Y_sample = Y_train[:, [idx]]

            # 1. Propagación
            self.propaga(X_sample)

            # 2. Retropropagación
            dEdW1, dEdb1, dEdW2, dEdb2, dEdW3, dEdb3 = self.backpropagation(X_sample, Y_sample)

            # 3. Actualización de pesos
            self.W3 -= self.eta * dEdW3
            self.b3 -= self.eta * dEdb3
            self.W2 -= self.eta * dEdW2
            self.b2 -= self.eta * dEdb2
            self.W1 -= self.eta * dEdW1
            self.b1 -= self.eta * dEdb1

    def test(self, X_test):
        results = []
        print(f"\n--- Probando Red ---")
        for i in range(X_test.shape[1]):
            x = X_test[:, [i]]
            pred = self.propaga(x)
            print(f"Entrada: {x.T} -> Salida Red: {pred.T[0]} (Redondeado: {np.round(pred.T[0])})")
            results.append(pred)
        return results

# ==========================================
# DATOS DE ENTRENAMIENTO (Tablas de Verdad)
# ==========================================
# Entradas (iguales para ambas): 00, 01, 10, 11
X_data = np.array([
    [0, 0, 1, 1],
    [0, 1, 0, 1]
])

# Salidas Deseadas NAND (1 si no son ambos 1)
Y_NAND = np.array([[1, 1, 1, 0]])

# Salidas Deseadas XOR (1 si son diferentes)
Y_XOR = np.array([[0, 1, 1, 0]])

# ==========================================
# RED NEURONAL 1: APRENDIENDO NAND
# ==========================================
print("\n>>> ENTRENANDO RED NAND <<<")
# Arquitectura: 2 entradas, 4 oculta1, 4 oculta2, 1 salida
nn_nand = RedNeuronalDosOcultas(nn_input=2, nn_hidden1=4, nn_hidden2=4, nn_output=1, eta=0.5)
nn_nand.train(X_data, Y_NAND, epochs=20000)
nn_nand.test(X_data)

# ==========================================
# RED NEURONAL 2: APRENDIENDO XOR
# ==========================================
print("\n>>> ENTRENANDO RED XOR <<<")
# XOR es más complejo, a veces requiere más épocas o neuronas
nn_xor = RedNeuronalDosOcultas(nn_input=2, nn_hidden1=8, nn_hidden2=8, nn_output=1, eta=0.5)
nn_xor.train(X_data, Y_XOR, epochs=30000)
nn_xor.test(X_data)